Import the USSCS database kernel

In [1]:
from db_kernel import manage
from db_kernel import user as user_kernel
#without login:
def add_user(username:str, password:str, twofa:bool):
    """
    Add a user to the database.
    If two factor authentication is enabled, will return qrcode image.
    """
    return manage().add_user(username, password, twofa)

class user:
    def __init__(self, username:str, passwort:str, twofa_key:str=None):
        self._user = user_kernel(username, passwort, twofa_key)
    
    def send_message(self, target:str, message:str, type:str='text'):
        """
        Send a message to a user.
        """
        return self._user.send_message(target, message, type)
    def get_messages(self, target:str, message_id:int=-1):
        """
        Get all messages from a user.
        id is the id of the last message you have.
        """
        return self._user.get_conversation(target, message_id)
    def get_users_with_unread_messages(self):
        """
        Get all users with unread messages.
        """
        return self._user.get_unread_messages()


In [2]:
add_user('test', 'test', False)
add_user('test2', 'test', False)
user('test', 'test').send_message('test2', 'test')
print(user('test2', 'test').get_messages('test'))

[('test', b'test', '16:50 06/04/23', 0, 'text')]
